# 2.6 Segmentación Básica

**Resumen Ejecutivo**: Agrupa clientes y productos en categorías (alto/medio/bajo margen, ticket sobre promedio, RFM) para priorizar acciones comerciales y de producto.

---

## 🎯 Objetivos de Aprendizaje
- Clasificar productos/clientes con `CASE` basado en métricas (margen, ticket, frecuencia).
- Construir segmentaciones RFM simplificadas (Recency, Frequency, Monetary).
- Combinar métricas y etiquetas para análisis multi-dimensional.

## 🧰 Prerrequisitos
- Haber completado `05_kpis_avanzados.ipynb`.
- Dominar agregaciones, `CASE`, y CTEs.
- Comprender distribuciones y umbrales de negocio.

---

## 📚 Conceptos Clave
- **Categorización**: `CASE` para etiquetar (ALTO/MEDIO/BAJO) basado en umbrales.
- **RFM**: Recency (última compra), Frequency (días activos), Monetary (valor acumulado).
- **Segmentación multi-nivel**: combinar dimensiones (segmento + categoría margen).
- **Separación métrica/etiqueta**: calcular primero, etiquetar después.

In [ ]:
-- Segmento vs volumen de unidades
SELECT c.segmento, SUM(fv.cantidad) AS unidades
FROM dbo.fact_ventas fv
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
GROUP BY c.segmento;

In [ ]:
-- Clasificación simple de productos por margen (alto, medio, bajo)
SELECT producto_id, nombre,
       CASE WHEN (precio_unitario - costo_unitario) >= 30 THEN 'ALTO'
            WHEN (precio_unitario - costo_unitario) >= 15 THEN 'MEDIO'
            ELSE 'BAJO' END AS categoria_margen
FROM dbo.dim_productos;

---

## 🧪 Ejercicios Guiados
- 🟢 Tabla derivada con unidades y margen por producto y categoría de margen (ALTO/MEDIO/BAJO).
- 🟠 Clientes con ticket promedio > ticket promedio global (JOIN vs subconsulta con CTEs).
- 🔴 Crear segmentación RFM simplificada (Recency, Frequency, Monetary) y asignar etiqueta compuesta.

In [ ]:
WITH prod AS (
  SELECT p.producto_id, p.nombre, (p.precio_unitario - p.costo_unitario) AS margen_u,
         CASE WHEN (p.precio_unitario - p.costo_unitario) >= 30 THEN 'ALTO'
              WHEN (p.precio_unitario - p.costo_unitario) >= 15 THEN 'MEDIO'
              ELSE 'BAJO' END AS categoria_margen
  FROM dbo.dim_productos p
)
SELECT categoria_margen, SUM(fv.cantidad) AS unidades, SUM(fv.cantidad*margen_u) AS margen_total
FROM prod
LEFT JOIN dbo.fact_ventas fv ON prod.producto_id = fv.producto_id
GROUP BY categoria_margen;

In [ ]:
WITH global AS (
  SELECT CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_global
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
), cliente AS (
  SELECT c.cliente_id, c.segmento,
         CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_cliente
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY c.cliente_id, c.segmento
)
SELECT cliente_id, segmento, ticket_cliente
FROM cliente CROSS JOIN global
WHERE ticket_cliente > ticket_global;

---

## ⚠️ Errores Comunes
- Etiquetar antes de limpiar datos (nulos/outliers distorsionan umbrales).
- Usar `CASE` anidados sin cubrir todos los rangos (valores inesperados quedan sin etiqueta).
- No separar cálculo de métrica y asignación de etiqueta (dificulta auditoría).
- Umbrales arbitrarios sin validación de negocio (segmentaciones inútiles).

---

## ✅ Conclusiones
- Segmentación con `CASE` facilita agrupaciones por umbrales de negocio.
- RFM combina tres dimensiones (recency, frequency, monetary) para priorizar clientes.
- Modularizar métricas en CTEs antes de etiquetar mejora claridad y reusabilidad.

---

## 🚀 Aplicación Práctica
- Campañas dirigidas a segmentos de alto margen o bajo ticket.
- Priorizar inventario y descuentos por categoría de producto.
- Dashboards RFM para equipos de CRM y retención.

---

## 💼 Perspectiva de Negocio
- Segmentación permite personalizar oferta (pricing, promociones, contenido).
- RFM identifica campe ones (alta recency/frequency/monetary) y at-risk (baja recency).
- Categorización automatizada reduce esfuerzo manual y acelera decisiones tácticas.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 2  
**Módulo**: 2.6 Segmentación Básica  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 Nota sobre IA: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](05_kpis_avanzados.ipynb) | [Siguiente ➡️](07_optimizar_consultas_basico.ipynb)
---
